# WRDS Data Queries

In [1]:
import pandas as pd
import numpy as np
import quandl
import wrds

In [2]:
BrianBoonstraPrivateKeys['wrds']['username']

'brianboonstra'

The following password-free login depends on having created a `~/.pgpass` file with with WRDS entries, as in 

```
wrds-pgdata.wharton.upenn.edu:9737:wrds:brianboonstra:<password>
```

In [3]:
db = wrds.Connection(
    wrds_username=BrianBoonstraPrivateKeys['wrds']['username'],
)

Loading library list...
Done


In [4]:
db.describe_table(library="trace", table='trade_summary')

Approximately 16362568 rows in trace.trade_summary.


,name,nullable,type
0,trans_dt,True,DATE
1,bond_sym_id,True,VARCHAR(14)
2,cusip_id,True,VARCHAR(9)
3,bsym,True,VARCHAR(12)
4,sub_prd_type,True,VARCHAR(5)
5,wis_fl,True,VARCHAR(1)
6,high_pr,True,DOUBLE_PRECISION
7,high_yld_sign_cd,True,VARCHAR(1)
8,high_yld,True,DOUBLE_PRECISION
9,low_pr,True,DOUBLE_PRECISION


In [5]:
db.list_tables(library="trace")

['absmasterfile',
 'cmomasterfile',
 'masterfile',
 'mbsmasterfile',
 'tbamasterfile',
 'trace',
 'trace_agency',
 'trace_agency_names',
 'trace_btds144a',
 'trace_btds144a_names',
 'trace_enhanced',
 'trace_enhanced_names',
 'trace_names',
 'trace_spds144a_abs',
 'trace_spds144a_abs_names',
 'trace_spds144a_cmo',
 'trace_spds144a_cmo_names',
 'trace_spds_abs',
 'trace_spds_abs_names',
 'trace_spds_cmo',
 'trace_spds_cmo_names',
 'trace_spds_mbs',
 'trace_spds_mbs_names',
 'trace_spds_tba',
 'trace_spds_tba_names',
 'trade_summary',
 'trade_summary_agency',
 'trade_summary_btds144a',
 'trade_summary_spds144a_abs',
 'trade_summary_spds144a_cmo',
 'trade_summary_spds_abs',
 'trade_summary_spds_cmo',
 'trade_summary_spds_mbs',
 'trade_summary_spds_tba']

In [6]:
db.raw_sql("SELECT COUNT(*) FROM trace.trace", )

,count
0,179125244


In [10]:
single_row = db.raw_sql(
"""
SELECT * 
FROM trace.trace
ORDER BY TRD_EXCTN_DT DESC
LIMIT 1
"""
    , )

In [11]:
single_row.transpose()

,0
cusip_id,Y9T10PX26
bond_sym_id,UOVE4359330
company_symbol,UOVE
bsym,BBG00CF2S0K9
trd_exctn_dt,2020-09-30
trd_exctn_tm,39565
sub_prd_type,CORP
trans_dt,2020-09-30
msg_seq_nb,15031
trc_st,M


In [18]:
some_traded_sql = """
SELECT 
 company_symbol, BOND_SYM_ID, cusip_id, RPTD_PR as price, 
 ASCII_RPTD_VOL_TX AS volm, FRMT_CD as quantity,
 TRD_EXCTN_DT as date, TRD_EXCTN_TM AS time_seconds
FROM trace.trace
WHERE TRD_EXCTN_DT > '2020-06-30' 
  AND cusip_id IS NOT NULL 

LIMIT 10
"""


In [19]:
some_trades = db.raw_sql(some_traded_sql, 
          date_cols=['date']
          )
some_trades

,company_symbol,bond_sym_id,cusip_id,price,volm,quantity,date,time_seconds
0,ABB,ABB3852125,00037BAB8,104.071,10000,A,2020-07-01,55045.0
1,ABB,ABB3852125,00037BAB8,104.255,20000,A,2020-07-01,60714.0
2,ABB,ABB3852142,00037BAC6,126.033,1000000,A,2020-07-01,56917.0
3,ABB,ABB4615653,00037BAE2,106.743,60000,A,2020-07-01,40517.0
4,ADT,ADT3991008,00101JAF3,100.263,50000,A,2020-07-01,34797.0
5,ADT,ADT3991008,00101JAF3,100.163,50000,A,2020-07-01,34797.0
6,ADT,ADT3991008,00101JAF3,100.163,50000,A,2020-07-01,34797.0
7,ADT,ADT3991008,00101JAF3,99.595,5000,A,2020-07-01,35182.0
8,ADT,ADT3991008,00101JAF3,100.635,50000,A,2020-07-01,38973.0
9,ADT,ADT3991008,00101JAF3,100.335,50000,A,2020-07-01,38973.0
